In [1]:
from carla import Benchmark
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
import pandas as pd
# Load Data 
dataframe = pd.read_csv('./census.csv')
continuous = dataframe.drop(columns=['target']).columns

dataset = CsvCatalog(file_path="census.csv",
                     continuous=continuous,
                     categorical=[],
                     immutables=[],
                     target='target')

In [3]:
#Model 

training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.2411957411957412, balance on test set 0.23965114850755437
Epoch 0/9
----------
train Loss: 0.5537 Acc: 0.7585

test Loss: 0.5514 Acc: 0.7603

Epoch 1/9
----------
train Loss: 0.5532 Acc: 0.7588

test Loss: 0.5509 Acc: 0.7603

Epoch 2/9
----------
train Loss: 0.5531 Acc: 0.7588

test Loss: 0.5525 Acc: 0.7603

Epoch 3/9
----------
train Loss: 0.5531 Acc: 0.7588

test Loss: 0.5508 Acc: 0.7603

Epoch 4/9
----------
train Loss: 0.5530 Acc: 0.7588

test Loss: 0.5531 Acc: 0.7603

Epoch 5/9
----------
train Loss: 0.5530 Acc: 0.7588

test Loss: 0.5507 Acc: 0.7603

Epoch 6/9
----------
train Loss: 0.5531 Acc: 0.7588

test Loss: 0.5507 Acc: 0.7603

Epoch 7/9
----------
train Loss: 0.5530 Acc: 0.7588

test Loss: 0.5507 Acc: 0.7603

Epoch 8/9
----------
train Loss: 0.5527 Acc: 0.7588

test Loss: 0.5522 Acc: 0.7603

Epoch 9/9
----------
train Loss: 0.5531 Acc: 0.7588

test Loss: 0.5514 Acc: 0.7603



In [4]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:2]

display(test_factual)

,Unnamed: 0,Age,Workclass,Education-Num,Marital Status,...,Capital Gain,Capital Loss,Hours per week,Country,target
0,0.000000,0.301370,0.875,0.8,0.666667,...,0.02174,0.0,0.397959,0.95122,False
1,0.000031,0.452055,0.750,0.8,0.333333,...,0.00000,0.0,0.122449,0.95122,False


In [5]:
hyperparams = {"loss_type": "BCE", "binary_cat_features": True}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
df_cfs = recourse_method.get_counterfactuals(test_factual)

display(df_cfs)

[INFO] Timeout - No Counterfactual Explanation Found [wachter.py wachter_recourse]


In [ ]:
def semantic(counterfactual, classification, causal_model):
    '''Compare Labels'''
    
    pass

''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)
